In [1]:
#1. Evaluate model 
#2. Swap and re-evaluate looping through all swappable variables and compute incremental contribution
#3. Parallelize

#1. Evaluate model
# Read modeling stack, coefficients

# Cast data into model matrix by X_ID

# Evaluate


In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
import time
sns.set()

# 1. Store an intermediate evaluation matrix for quick access


Instead of 

$y = A \beta$,

Can think of this as being split into two operations


1. $\tilde{A} = A \cdot diag(\beta)$

2. $y = rowsum(\tilde{A}) $



$\tilde{A} = A \cdot diag(\beta)$

# 2. Efficient adstock computation using matrix multiplication

Just a simple trick to save time.

# Notes:
1. Map-reduce implementation (pyspark)
2. Test specs:

    A. THD
    
    B. Dignity Health
    
    C. Citi Enterprise
    
    
3. Features to be added:

    A. This does not support HALOs yet - LHF
    
    B. It does not support non-uniform interactions - LHF


In [130]:
#2. Swap and re-evaluate looping through all swappable variables and compute incremental contribution
#define a singleModel class
import re
import pandas as pd
import numpy as np

def npdate2datetime(somedate):
    somedate_ts = (np.datetime64(somedate) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    somedate_dt = datetime.utcfromtimestamp(somedate_ts)
    return somedate_dt

def nearest(items, thisdate):
    return min(items, key=lambda x: abs(x - np.datetime64(thisdate)))

def suffixToAdstock(suffix):
    adstockSpec = int(suffix)
    persistence, adstockSpec = (adstockSpec%100)/100, adstockSpec//100
    peak, length = adstockSpec%10, adstockSpec//10
    return (length, peak, persistence)

#computeAdstock takes base var x and params (length, peak, persistence) and returns NormalizedAdstock vector|
def computeAdstockVector(x, params):
    #print('Length of var: ', x.shape)
    wk = 0
    l, p, r = params[0], params[1], params[2]
    y = []
    for i in range(len(x)):
        if wk <= l:
            if i < p and p > 0:
                y.append(i/p)
            else:
                y.append(np.power(r, i-p))
        else:
            y.append(0)
        wk+=1
    
    y = np.array(y)/np.sum(y)
    y_adstock = y
    if l>0:
        x = np.insert(x, l+1, x[l])
        y = np.insert(y, l+1, 0)
    if p==0:
        x=np.insert(x, 0, 0)
        y=np.insert(y, 0, 0)
    #print('Adstock Vector: \n', y)
    #print('Adstock Vector, Raw Adstock Vector, Lengths: ', y.shape, y_adstock.shape)
    return y_adstock

#takes in base var and normalized adstock values and returns transformed var
def computeAdstockedVar(yvar, asvals_53wk):
    adstock_matrix = np.zeros((len(yvar), len(yvar)))
    for i in range(len(yvar)):
        for j in range(len(asvals_53wk)):
            if (i+j)<len(yvar):
                adstock_matrix[i+j, i] = asvals_53wk[j]
            else: break
    return np.dot(adstock_matrix, yvar)

def invertAdstockedVar(yvar, asvals_53wk):
    adstock_matrix = np.zeros((len(yvar), len(yvar)))
    for i in range(len(yvar)):
        for j in range(len(asvals_53wk)):
            if (i+j)<len(yvar):
                adstock_matrix[i+j, i] = asvals_53wk[j]
            else: break
    #print('Adstock Matrix: ', adstock_matrix)
    return np.dot(np.linalg.pinv(adstock_matrix), yvar)

In [138]:
class singleModel():
    def __init__(self, estack, coeffs, ocome, thisxid, fbase='FloatingBase'):
        self.modelStack = estack
        self.modelStack['X_DT'] = pd.to_datetime(self.modelStack['X_DT'])
        self.startTime = self.modelStack['X_DT'].min()
        self.endTime = self.modelStack['X_DT'].max()
        self.coeffs = coeffs
        if 'X_ID' in list(self.coeffs['Variable']):
            self.intercept = self.coeffs.loc[self.coeffs['Variable']=='X_ID', 'Coefficient']
        else:
            self.intercept = 0.0
        self.coeffMeans = np.array(self.coeffs.loc[self.coeffs['Variable']!='X_ID', 'Coefficient'])
        self.coeffMeans = np.append(self.coeffMeans, self.intercept)
        self.xid = thisxid
        self.outcomeName = ocome
        self.modeledVars = list(self.coeffs.loc[self.coeffs['Variable']!='X_ID', 'Variable'])
        self.tps = [var for var in self.modeledVars if var.startswith('M_')]
        self.fbname = fbase
        
        self.transformationType, self.varsByTransformation = self.getTransformNames(self.modeledVars)
    
    def getTransformNames(self, varnames):
        self.transformationType={var:'NA' for var in varnames}
        self.varsByTransformation = {}
        for var in varnames:
            varSegments = var.split('_')
            adstockSegment = [seg for seg in varSegments if (len(seg)>=4 and len(seg)<=5 and seg.isdigit())]
            if adstockSegment:
                self.transformationType[var] = adstockSegment[0]
                if adstockSegment[0] in self.varsByTransformation.keys():
                    self.varsByTransformation[adstockSegment[0]].append(var)
                else:
                    self.varsByTransformation[adstockSegment[0]] = [var]
            elif 'LOGX' in var.split('_'):
                self.transformationType[var] = 'LOGX'
                if 'LOGX' in self.varsByTransformation.keys():
                    self.varsByTransformation['LOGX'].append(var)
                else:
                    self.varsByTransformation['LOGX']=[var]
            elif 'LOGC' in var.split('_'):
                self.transformationType[var] = 'LOGC'
                if 'LOGC' in self.varsByTransformation.keys():
                    self.varsByTransformation['LOGC'].append(var)
                else:
                    self.varsByTransformation['LOGC'] = [var]
            else:
                self.transformationType[var] = 'Raw'
                if 'Raw' in self.varsByTransformation.keys():
                    self.varsByTransformation['Raw'].append(var)
                else:
                    self.varsByTransformation['Raw'] = [var]
                    
        return self.transformationType, self.varsByTransformation

    
    def evaluate(self, stack = None, start=None, end=None): #returns evaluation
        if start==None:
            start = self.startTime
        if end == None:
            end = self.endTime
        if stack is None:
            stack=self.modelStack
        y = np.array(stack[self.outcomeName])
        X = stack.loc[(stack['X_DT']>=start) & (stack['X_DT']<=end), self.modeledVars].values
        b = np.array(self.coeffMeans)
        fb = stack[self.fbname]
        oneMatrix = np.ones((X.shape[0], X.shape[1]+1))
        oneMatrix[:, :-1] = X
        X = oneMatrix
        del oneMatrix
        self.evalMatrix = X.dot(np.diag(b))
        self.evaluation = np.exp(X.dot(b)+fb)
        return self.evaluation

    def getRawContribution(self, varnames, start=None, end=None):
        if start == None:
            start = self.startTime
        if end == None:
            end = self.endTime
        varIndices = [self.modeledVars.index(varname) for varname in varnames]
        evals = self.evaluate(start=start, end=end)
        return np.multiply(evals, (1-1/np.exp(np.sum(self.evalMatrix[:, varIndices], axis=1))))
    
    def getDecompAdjustment(self, start=None, end=None):
        if start == None:
            start = self.startTime
        if end == None:
            end = self.endTime
        soloTotalContribution = sum([self.getRawContribution([var], start, end).sum() for var in self.tps])
        comboTotalContribution = self.getRawContribution(self.tps, start, end).sum()
        return comboTotalContribution/soloTotalContribution
    
    def getDecomps(self, varnames, start=None, end=None):
        if start == None:
            start = self.startTime
        if end == None:
            end = self.endTime
        rawContribution = self.getRawContribution(varnames, start, end)
        tpContribution = self.getRawContribution(varnames)
        
    def getSwapAdjustment(self, start=None, end=None):
        if start == None:
            start = self.startTime
        if end == None:
            end = self.endTime
        return 1
        
    
    def getRawSwap(self, varname, focusStart, focusEnd=None):
        if focusEnd==None:
            focusEnd = self.modelStack['X_DT'].max()
        #1. swap out var[focusStart:focusEnd] with var[focusStart-1yr:focusEnd-1yr]
        stack = self.modelStack[(self.modelStack['X_DT']>=focusStart) & (self.modelStack['X_DT']<=focusEnd)]
        swapStack = stack.copy(deep=True)
        focusStart_ts =  (np.datetime64(focusStart) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
        focusStart_dt = datetime.utcfromtimestamp(focusStart_ts)
        swapStart = np.datetime64(focusStart_dt.replace(year=(focusStart_dt.year)-1))
        print('raw swap start:', swapStart)
        swapStart = nearest(np.array(self.modelStack['X_DT'].unique()), swapStart)
        print('stacked swap start:', swapStart)
        swapEnd = focusEnd.replace(year=(focusEnd.year)-1)
        swapEnd = nearest(np.array(self.modelStack['X_DT'].unique()), swapEnd)
        print(len(swapStack))
        print(len(np.array(self.modelStack.loc[(self.modelStack['X_DT']>=swapStart)&(self.modelStack['X_DT']<=swapEnd)])))
        swapStack.loc[:, varname] = np.array(self.modelStack.loc[(self.modelStack['X_DT']>=swapStart)&(self.modelStack['X_DT']<=swapEnd), varname])
        
        #2. compute raw swap
        swappedEvals = self.evaluate(stack=swapStack, start=stack['X_DT'].min(), end=stack['X_DT'].max()).sum()
        evals = self.evaluate(stack=stack, start=stack['X_DT'].min(), end=stack['X_DT'].max()).sum()
        return evals-swappedEvals
        
    #just invert the adstock to get halfInverted transformed vars - scalars intact. 
    #vars just unlogged and inverse-Adstocked
    #can be replaced with scalars and raw stacked vars to be more efficient
    def getHalfBakedStack(self):
        self.halfBakedStack = self.modelStack.copy(deep=True)
        for transformType in self.varsByTransformation.keys():
            if transformType.isdigit():
                for var in self.varsByTransformation[transformType]:
                    self.halfBakedStack.loc[:, var] = np.exp(self.halfBakedStack[var])
                    self.halfBakedStack.loc[:, var] = self.halfBakedStack.loc[:, var] - 1
                    #print('Debug: ', self.halfBakedStack.loc[:, var])
                    adstockParams = suffixToAdstock(transformType)
                    #print('Length, Peak, Persistence: ', adstockParams)
                    adstockVector = computeAdstockVector(np.array(self.halfBakedStack[var]), adstockParams)
                    self.halfBakedStack.loc[:, var] = invertAdstockedVar(np.array(self.halfBakedStack[var]), adstockVector)
                    newVar = '_'.join(var.split('_')[:-1]+['HFBK'])
                    self.halfBakedStack.rename(columns={var:newVar}, inplace=True)
        return self.halfBakedStack
        
    def induceZeros(self, varnames, start=None, end=None):
        if start==None:
            start = self.startTime
        if end==None:
            end = self.endTime
        #paste code for adstock*indicator_matrix*inverse_adstock*var here
        #note that Adstocked(var)(t) is only dependent on var(s) for s<=t. So if 
        for var in varnames:
            varHfbk = '_'.join(var.split('_')[:-1]+['HFBK'])
            suffix = var.split('_')[-1]
            if varHfbk in list(self.halfBakedStack):
                self.halfBakedStack[var] = self.halfBakedStack[varHfbk]
                self.halfBakedStack.loc[(self.halfBakedStack['X_DT']>=start)&(self.halfBakedstack['X_DT']<=end), var] = 0
                if suffix.isdigit():
                    adstockParams = suffixToAdstock(suffix)
                    adstockVector = computeAdstockVector(np.array(self.halfBakedStack[var]), adstockParams)
                    self.halfBakedStack.loc[:, var] = computeAdstockedVar(np.array(self.halfBakedStack[var]), adstockVector)
                else:
                    self.halfBakedStack.loc[(self.halfBakedStack['X_DT']>=start)&(self.halfBakedstack['X_DT']<=end), var] = 1
            else:
                print('Error. ', var, ' Not in stack.')
        return 0

In [140]:
start = time.time()
xidmodel = singleModel(estack_xid1, coeffs_xid1, outcome, 'XRT')
#pprint(xidmodel.transformationType)
#pprint(xidmodel.varsByTransformation)
xidmodel.getHalfBakedStack()
evals = xidmodel.evaluate()
decompDict = {var:xidmodel.getRawContribution([var]).sum() for var in list(xidmodel.modeledVars) if var.startswith('M_')}
#print(decompDict)
print(time.time() - start, ' seconds to run partial MOR for single XID')

C:\Users\nnambira\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0.5000500679016113  seconds to run partial MOR for single XID


In [9]:
xidmodel.getRawSwap('M_NP_N_PRD_IMP_8030', focusStart = focusStart)

NameError: name 'focusStart' is not defined

In [4]:
estack = pd.read_csv('Documents/MPI/troweInbCalls/estack.csv')
print(estack.head())

coeffs = pd.read_csv('Documents/MPI/troweInbCalls/coeffs.csv')
#coeffs['X_ID'] = ['|'.join([val1, val2, val3]) for (val1, val2, val3) in zip(list(coeffs['InteractionValue3']), list(coeffs['InteractionValue1']), list(coeffs['InteractionValue2']))]
coeffs['X_ID'] = coeffs['InteractionValue1']
#get fixed effect
#coeffs.loc[coeffs['Type']=='FE', 'X_ID'] = coeffs['InteractionValue1']


fbase = pd.read_csv('Documents/MPI/troweInbCalls/floatingBase.csv')
fbase.drop('RowIndex', axis=1, inplace=True)

fbase['X_DT']=pd.to_datetime(fbase['X_DT'])
estack['X_DT'] = pd.to_datetime(estack['X_DT'])

estack = pd.merge(estack, fbase, on=['X_ID', 'X_DT'])

         X_DT X_ID  D_HOLDOUT X_PRD  D_PR_UV  D_VALID_BOTH  D_VALID_XRT  \
0  2014-01-06  RET          0   RET        0             1            0   
1  2014-01-13  RET          0   RET        0             1            0   
2  2014-01-20  RET          0   RET        0             1            0   
3  2014-01-27  RET          0   RET        0             1            0   
4  2014-02-03  RET          0   RET        0             1            0   

   D_VALID_RET  log_C_CAL_NEW_INB_CNT  C_GQV_RET_TRP_GIX_LOGX  \
0            1               7.459915                0.092082   
1            1               7.474772                0.086673   
2            1               7.406103                0.079475   
3            1               7.499977                0.077930   
4            1               7.361375                0.071162   

             ...             M_SM_FB_IMP_8120  M_SM_FB_IMRE_IMP_8120  \
0            ...                     0.033718                    0.0   
1            .

In [5]:
estack_xid1 = estack[estack['X_ID']=='XRT']
coeffs_xid1 = coeffs[coeffs['X_ID']=='XRT']
outcome = coeffs['DepVar'].unique()[0]

In [99]:
focusStart = estack['X_DT'].unique()[-13]
#focusStart =  (focusStart - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
focusStart_ts =  (focusStart - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
focusStart = datetime.utcfromtimestamp(focusStart_ts)
np.datetime64(focusStart)
#blahStart = focusStart.replace(year=focusStart.year)

C:\Users\nnambira\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until


numpy.datetime64('2017-01-02T00:00:00.000000')

In [135]:
evals = xidmodel.evaluate()
decompDict = {var:xidmodel.getRawContribution([var]).sum() for var in list(xidmodel.modeledVars) if var.startswith('M_')}
print(decompDict)

{'M_MG_N_BRD_IMP_8140': 13.254581730729342, 'M_MG_N_PRD_IMP_8140': 21.232735691891904, 'M_NP_N_BRD_IMP_8030': 37.201963462573985, 'M_NP_N_PRD_IMP_8030': 14.02115113538105, 'M_OD_IMP_8020': 29.214871684221762, 'M_OD_PG_BRD_CLK_8020': 0.38878820709256684, 'M_OD_PG_PRD_CLK_8020': 18.2803634424547, 'M_OD_SS_BRD_CLK_8020': 3.0217599488163525, 'M_OD_SS_PRD_CLK_8020': 18.693517658022554, 'M_OV_RET_IMP_8010': 0.9965775410842174, 'M_OV_XRT_IMP_8010': 6.8389761827815025, 'M_PS_BRD_CLK_8010': 56.47415366435719, 'M_PS_PRD_CLK_8010': 74.42101821072008, 'M_RD_BRD_IMP_8160': 56.252984560300575, 'M_RD_PRD_IMP_8160': 0.0, 'M_SM_FB_IMP_8120': 6.4045147303754595, 'M_SM_FB_IMRE_IMP_8120': 0.8533027129662888, 'M_SM_IG_IMP_8120': 0.1269610679497038, 'M_SM_IG_IMRE_IMP_8120': 1.7078733584524621, 'M_SM_LI_IMP_8120': 2.443244136370027, 'M_SM_TW_IMP_8120': 5.0849909768507375, 'M_SM_TW_IMRE_IMP_8120': 0.9008757667350826, 'M_TV_BRD_GRP_13280': 142.97427700318713}


In [114]:
xidmodel.getDecompAdjustment()

0.9730149246881088

In [ ]:
xidmodel.rawDecomp([var for var in list(xidmodel.modeledVars) if var.startswith('M_')]).sum()

In [ ]:
sum(decompDict.values())

In [29]:
date = estack['X_DT'].min()

In [31]:
date

Timestamp('2014-01-06 00:00:00')

In [34]:
newdate=date.replace(year=(date.year)-1)
newdate

Timestamp('2013-01-06 00:00:00')

In [ ]:
coeffs.head()

In [ ]:
estack

In [ ]:
list(coeffs_xid1)

In [ ]:
estack.head(4)

In [ ]:
%matplotlib notebook
plt.plot(np.exp(np.array(estack_xid1[outcome])))
plt.plot(np.array(evals))

In [ ]:
blah.shape

In [ ]:
from multiprocessing import Pool

def getResults(xid):
    global estack
    global coeffs
    estack_xid1 = estack[estack['X_ID']==xid]
    coeffs_xid1 = coeffs[coeffs['X_ID']==xid]
    outcome = coeffs['DepVar'].unique()[0]
    xidmodel = singleModel(estack_xid1, coeffs_xid1, outcome, xid)

    evals = xidmodel.evaluate()
    decompDict = {var:xidmodel.rawDecomp([var]).sum() for var in list(xidmodel.modeledVars) if var.startswith('M_')}
    return evals, decompDict

In [ ]:
with Pool(2) as p:
    print(p.map(getResults, ['XRT', 'RET']))